Purpose: tracklets and bounding boxes are in one place after extracting the .gz file written during the tacking session. Now, we run keypoint inference on the input video using the tracklets we have extracted.

In [1]:
import json
from sparrow_datums import Boxes
import numpy as np
import imageio
from tqdm import tqdm
import cv2
import os
from pathlib import Path
# import torch

In [2]:
from speed_trapv3.keypoints.model import SegmentationModel
from speed_trapv3.keypoints.config import Config as KeyConfig
from speed_trapv3.keypoints.dataset import crop_and_resize,process_keypoints, keypoints_post_inference_processing
from speed_trapv3.tracking.tracking import get_video_properties, transform_image, write_to_json

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
video_path = "/code/data/datasets/common_hall/source_video/25_resampled_vid.mp4"
save_path = video_path = "/code/data/datasets/common_hall/source_video/25_resampled_vid.mp4"
save_path = '/code/data/datasets/common_hall/tracking_inference_frame_lib'

In [4]:
# ! rm -rf /code/data/datasets/common_hall/tracking_inference_frame_lib/*

In [5]:
# fps, n_frames = get_video_properties(video_path)
# reader = imageio.get_reader(video_path)
# slug = "hard_coded"
# for i in tqdm(range(n_frames)):
#     img = reader.get_data(i)
#     img = cv2.rectangle(
#         img, (450, 200), (1280, 720), thickness=5, color=(0, 255, 0))
#     filename = str(i) + ".jpg"
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     cv2.imwrite(os.path.join(save_path, filename), img)

In [6]:
keypoint_model = SegmentationModel().eval().cuda()
keypoint_model.load(KeyConfig.trained_model_path)

<All keys matched successfully>

In [7]:
f = open('/code/data/datasets/tracking/predictions/hard_coded_vehicle.json')
data = json.load(f)
for obj in data['annotations']:
    obj_id = obj['id']
    frame_start, frame_end = obj['segments'][0]
    boxes = obj['frames']
    for frame_idx in range(frame_start, frame_end):
            # img = imageio.imread(Path(save_path)/f'{frame_idx}.jpg')
            img = cv2.imread(str(Path(save_path)/f'{frame_idx}.jpg'))
            box = boxes[str(frame_idx)]['bounding_box']
            x = box['x']
            y = box['y']
            w = box['w']
            h = box['h']
            box = Boxes([x, y, w, h]).to_tlbr()
            x1, y1, x2, y2 = box
            roi_w = x2 - x1
            roi_h = y2 - y1
            roi_resized = crop_and_resize(box, img, KeyConfig.image_crop_size[0],KeyConfig.image_crop_size[1])
            roi_resized_w, roi_resized_h = roi_resized.size
            # x = image_transform(roi_resized)
            # x = torch.unsqueeze(x, 0).cuda()
            # keypoints = keypoint_model(x)['keypoints'][0].detach().cpu().numpy()
            keypoints = keypoint_model(roi_resized)
            keypoints = keypoints_post_inference_processing(
                keypoints, roi_resized_w, roi_resized_h, roi_w, roi_h, x1, y1
            )

TypeError: Can't convert object to 'str' for 'filename'

In [ ]:
KeyConfig.image_crop_size[0]